<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Induction Gem Guide

## Authors: Samuel Cupp, Leo Werneck, and Zach Etienne

<font color='red'>**This module is currently under development**</font>

### Required and recommended citations:
* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).
* **(Recommended)** [Del Zanna, Bucciantini, and Londrillo. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics (2003)](https://arxiv.org/abs/astro-ph/0210618).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [**Introduction**](#introduction)
1. [**The General Design**](#design)
    1. [*Flux Terms*](#flux_terms)
    1. [*Gauge Terms*](#gauge_terms)
1. [**Usage Example**](#example)

<a id='introduction'></a>

# Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

The `Induction` gem provides several functions assist with computing the right-hand sides (RHS) for the induction equation. These functions are split between those which are naturally computed in a single direction (flux contributions to $A_i^{\rm RHS}$) and those which are direction-independent ($\tilde{\Phi}^{\rm RHS}$ and gauge contributions to $A_i^{\rm RHS}$).

<a id='design'></a>

# The General Design \[Back to [top](#toc)\]

$$\label{design}$$

To compute the RHS, several considerations have to be made. First, we need to decide if we have staggered variables. Currently, all `GRHayL` functions assume that the vector potential $A_i$ is staggered in the perpendicular directions (i.e. $A_x$ is staggered in the y,z directions and $A_y$ is staggered in the x,z directions). Due to the implementation of the HLL flux, `GRHayL`'s notation also assumes the presence of a $B^i$ staggered in one direction (i.e. $B^x$ is staggered in the x direction and $B^y$ is staggered in the y direction). However, note that since all `GRHayL` functions only take stencils of variables, `GRHayL` is entirely unaware of the source of the input $B_{\rm stagger}$. The scalar potential $\tilde{\Phi}$ is fully staggered to $(i+\frac{1}{2},j+\frac{1}{2},k+\frac{1}{2})$.

TODO: give explicit definition of $B$, since we do differ from e.g. Noble2D by factor

<a id='flux_terms'></a>

## Flux Terms \[Back to [top](#toc)\]

$$\label{flux_terms}$$

`GRHayL` currently only implements the Harten, Lax, and van Leer (HLL) flux method (see e.g. [Del Zanna, Bucciantini & Londrillo](https://arxiv.org/abs/astro-ph/0210618)). This is provided with the `ghl_HLL_2D_flux` function:

```
double ghl_HLL_2D_flux(const HLL_2D_vars *restrict vars)
```
which directly returns the RHS contribution. The function automatically computes any flux direction, as all direction-dependent data is handled by properly setting the `HLL_2D_vars` struct. The `HLL_2D_vars` struct is more complicated than most structs due to the cross-product terms that require different coordinates depending on the component of the RHS being computed. As such, it is highly recommended that users read the [HLL flux](ghl_HLL_2D_flux.ipynb) documentation before using. This function requires multiple reconstructions of the velocities to get to the staggered gridpoint of $A_i$, and improperly packing the struct will cause unexpected behavior.

<a id='gauge_terms'></a>

## Gauge Terms \[Back to [top](#toc)\]

$$\label{gauge_terms}$$

`GRHayL` provides functions to facilitate computation of $\tilde{\Phi}^\mathrm{RHS}$ and the gauge terms of $A_i^\mathrm{RHS}$. Both of these RHS values require stencils of quantities that have to be computed. To make the gem infrastructure-agnostic (and to help with OMP parallelism), the two functions `interpolate_for_A_gauge_rhs` and `calculate_phitilde_rhs` are provided to assist with computing these RHS values.

The first function
```
void ghl_interpolate_for_induction_rhs(
      const metric_quantities metric_stencil[2][2][2],
      const double psi_stencil[2][2][2],
      const double Ax_stencil[3][3][3],
      const double Ay_stencil[3][3][3],
      const double Az_stencil[3][3][3],
      const double phitilde,
      induction_interp_vars *restrict interp_vars)
```
takes stencils of the metric quantities and $A_i$ and computes intermediaries interpolated to the staggered locations of $\tilde{\Phi}$ and $A_i$.

The required input variables are the inverse BSSN metric $\tilde{g}^{ij}$, BSSN variable $\psi$, lapse, shift, vector potential $A_i$, and scalar potential $\tilde{\Phi}$. The metric quantities are passed via an array of `metric_quantities` structs, and this can be easily filled by calling the `initialize_metric` function for each array element.

The output variables are the lapse, shift, and $(\alpha \Phi - \beta^j A_j)$ interpolated to $(i+\frac{1}{2},j+\frac{1}{2},k+\frac{1}{2})$, and $\alpha \sqrt{\gamma} A^i$ interpolated to the location of $A_i$.

The [ghl_interpolate_for_induction_rhs](ghl_interpolate_for_induction_rhs.ipynb) function documentation provides more information about using this function.

The gauge contribution to $A_i^\mathrm{RHS}$ can be directly computed from the interpolated quantity $\alpha \Phi - \beta^j A_j$ using a finite difference algorithm.

The second function
```
double ghl_calculate_phitilde_rhs(
      const double Lorenz_damping_factor,
      A_gauge_rhs_vars *restrict vars)
```
takes stencils of the intermediate quantities and returns $\tilde{\Phi}^\mathrm{RHS}$. While users are free to compute these intermediate quantities in their own way, it is highly recommended to use the two functions together. The [ghl_calculate_phitilde_rhs](ghl_calculate_phitilde_rhs.ipynb) function documentation provides more information about using this function.

<a id='example'></a>

# Usage Example \[Back to [top](#toc)\]
$$\label{example}$$

This example is taken from the `GRHayLMHD` implementation in [evaluate_MHD_rhs.c](https://github.com/SamuelCupp/GRHayL_beta/blob/main/implementations/GRHayLMHD/src/evaluate_MHD_rhs.c) and [evaluate_phitilde_and_A_gauge_rhs.c](https://github.com/SamuelCupp/GRHayL_beta/blob/main/implementations/GRHayLMHD/src/evaluate_phitilde_and_A_gauge_rhs.c). While this function computes all the RHS variables, only the outline of steps for the induction equation will be included here. All reconstructed operations are included, even those only needed for the GRHD RHS functions to make implementation easier. Reconstructed variables will be denoted with an $R$ followed by the coordinates they've been reconstructed in. Note that this implementation reconstructs to $m-\frac{1}{2}$, while the magnetic components are needed at $m+\frac{1}{2}$. This leads to additional indexing complications for computing $A_i^\mathrm{RHS}$. As this choice is inherited from IlliniosGRMHD, we leave it as is until an example code implements a simpler version of this process.

Also, this code has precomputed values for staggered $B^i$, which we denote here as $\underline{B}^i$. These grid functions are staggered in the coordinate $i$ (e.g. $\underline{B}^x$ lives at the point $(i+\frac{1}{2},j,k)$).

## Steps

### First reconstruction direction ($x$)

1. Reconstruct $\rho_b$, $P$, $v^i$, $B^x$, $B^z$, $\underline{B}^y$ in the $x$ direction using `ghl_ppm`
   1. We can skip one reconstruction by using $B^x_{Rx}(i,j,k) = \underline{B}_x(i-1,j,k)$ (the minus one accounts for $\underline{B}_x$ being at staggered point $i+\frac{1}{2}$ while reconstruction gives $i-\frac{1}{2}$
   1. This gives us e.g. $P^{Rx}$ at the point $(i-\frac{1}{2},j,k)$ and $\underline{B}^y_{Rx}$ at the point $(i-\frac{1}{2},j+\frac{1}{2},k)$
1. Compute the characteristic speeds in the $x$ direction using `ghl_calculate_characteristic_speed_dirn0`

### Second reconstruction direction ($y$)

1. Reconstruct $v^x_{Rx}$ and $v^y_{Rx}$ in the $y$ direction using `ghl_ppm_no_rho_P`
   1. This gives us $v^i_{Rxy}$ at the point $(i-\frac{1}{2},j-\frac{1}{2},k)$
1. Reconstruct $\rho_b$, $P$, $v^i$, $B^x$, $B^z$, $\underline{B}^x$, $\underline{B}^z$ in the $y$ direction using `ghl_ppm`
   1. We can skip one reconstruction by using $B^y_{Ry}(i,j,k) = \underline{B}_y(i,j-1,k)$ (the minus one accounts for $\underline{B}_y$ being at staggered point $j+\frac{1}{2}$ while reconstruction gives $j-\frac{1}{2}$
   1. This gives us e.g. $P^{Ry}$ at the point $(i,j-\frac{1}{2},k)$ and $\underline{B}^x_{Ry}$ at the point $(i+\frac{1}{2},j-\frac{1}{2},k)$
1. Compute the characteristic speeds in the $y$ direction using `ghl_calculate_characteristic_speed_dirn1`
1. Compute $A_z^\mathrm{RHS}$ using `ghl_HLL_flux`
   1. At this point, we have the following variables:
      1. $v^i_{Rxy}$ at $(i-\frac{1}{2},j-\frac{1}{2},k)$
      1. $\underline{B}^x_{Ry}$ at $(i+\frac{1}{2},j-\frac{1}{2},k)$
      1. $\underline{B}^y_{Rx}$ at $(i-\frac{1}{2},j+\frac{1}{2},k)$
      1. $c^x_{min}$ at $(i-\frac{1}{2},j,k)$<a name="cspeed_ref"></a>[<sup>[1]</sup>](#cspeed_note)
      1. $c^y_{min}$ at $(i,j-\frac{1}{2},k)$<a name="cspeed_ref"></a>[<sup>[1]</sup>](#cspeed_note)
   1. These need to be properly indexed to get all of them to the $+\frac{1}{2}$ point, meaning adding 1 to any indices with the minus.
   

### Third reconstruction direction ($z$)

1. Reconstruct $v^y_{Ry}$ and $v^z_{Ry}$ in the $z$ direction using `ghl_ppm_no_rho_P`
   1. This gives us $v^i_{Ryz}$ at the point $(i,j-\frac{1}{2},k-\frac{1}{2})$
1. Reconstruct $\rho_b$, $P$, $v^i$, $B^x$, $B^y$, $\underline{B}^x$, $\underline{B}^y$ in the $z$ direction using `ghl_ppm`
   1. We can skip one reconstruction by using $B^z_{Ry}(i,j,k) = \underline{B}_z(i,j,k-1)$ (the minus one accounts for $\underline{B}_z$ being at staggered point $k+\frac{1}{2}$ while reconstruction gives $k-\frac{1}{2}$
   1. This gives us e.g. $P^{Rz}$ at the point $(i,j,k-\frac{1}{2})$ and $\underline{B}^x_{Rz}$ at the point $(i+\frac{1}{2},j,k-\frac{1}{2})$
1. Compute the characteristic speeds in the $z$ direction using `ghl_calculate_characteristic_speed_dirn2`
1. Compute $A_x^\mathrm{RHS}$ using `ghl_HLL_flux`
   1. At this point, we have the following variables:
      1. $v^i_{Ryz}$ at $(i,j-\frac{1}{2},k-\frac{1}{2})$
      1. $\underline{B}^z_{Ry}$ at $(i,j-\frac{1}{2},k+\frac{1}{2})$
      1. $\underline{B}^y_{Rz}$ at $(i,j+\frac{1}{2},k-\frac{1}{2})$
      1. $c^y_{min}$ at $(i,j-\frac{1}{2},k)$<a name="cspeed_ref"></a>[<sup>[1]</sup>](#cspeed_note)
      1. $c^z_{min}$ at $(i,j,k-\frac{1}{2})$<a name="cspeed_ref"></a>[<sup>[1]</sup>](#cspeed_note)
   1. These need to be properly indexed to get all of them to the $+\frac{1}{2}$ point, meaning adding 1 to any indices with the minus.


### Fourth reconstruction direction ($x$ again)

1. Reconstruct $\underline{B}^z$, $v^x_{Rz}$ and $v^z_{Rz}$ in the $x$ direction using `ghl_ppm_no_rho_P`
   1. This gives us $v^i_{Ryz}$ at the point $(i,j-\frac{1}{2},k-\frac{1}{2})$ and $\underline{B}^z_{Rx}$ at the point $(i-\frac{1}{2},j,k+\frac{1}{2})$
1. Compute the characteristic speeds in the $z$ direction using `ghl_calculate_characteristic_speed_dirn2`
1. Compute $A_y^\mathrm{RHS}$ using `ghl_HLL_flux`
   1. At this point, we have the following variables:
      1. $v^i_{Rzx}$ at $(i-\frac{1}{2},j,k-\frac{1}{2})$
      1. $\underline{B}^z_{Rx}$ at $(i-\frac{1}{2},j,k+\frac{1}{2})$
      1. $\underline{B}^x_{Rz}$ at $(i+\frac{1}{2},j,k-\frac{1}{2})$
      1. $c^x_{min}$ at $(i-\frac{1}{2},j,k)$<a name="cspeed_ref"></a>[<sup>[1]</sup>](#cspeed_note)
      1. $c^z_{min}$ at $(i,j,k-\frac{1}{2})$<a name="cspeed_ref"></a>[<sup>[1]</sup>](#cspeed_note)
   1. These need to be properly indexed to get all of them to the $+\frac{1}{2}$ point, meaning adding 1 to any indices with the minus.

### Compute direction-independent RHS terms

1. Compute interpolated quantities using `ghl_interpolate_for_induction_rhs`.
2. Compute the gauge terms of $A_i^\mathrm{RHS}$ with a finite difference algorithm using the computed quantity interp_vars->alpha_Phi_minus_betaj_A_j_interp
3. Compute $\tilde{\Phi}^\mathrm{RHS}$ using `ghl_calculate_phitilde_rhs`

# Footnotes
<a name="cspeed_note"></a>1. [^](#cspeed_ref) Note that the characteristic speeds are not actually at the same point. We could calculate these values at the correct location by computing the characteristic speeds using the doubly reconstructed velocities. However, this would introduce many more loops over the `calculate_characteristic_speed_dirn*` functions since we need these $c^i$ for the GRHD RHS calculations anyways. In addition, we would either need to introduce more grid functions or reconstruct the velocities repeatedly to get everything we need for every RHS calculation. Yuk Tung Liu discussed this point with M. Shibata, who found that the effect of this offset is negligible.